In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random
from copy import deepcopy

def plot(function, x1=-10, x2=10, step=1, x = None):
    x = [x for x in range(x1, x2 + step, step)] if x is None else x
    plt.plot(x, function(np.array(x)))
    plt.show()

def mul_matrix_by_vector(matrix, vector):
    assert len(matrix[0]) == len(vector)
    return np.array([sum(matrix[i][j] * vector[j] for j in range(len(vector))) for i in range(len(matrix))])

def mul_matrix(matrix1, matrix2):
    assert len(matrix1[0]) == len(matrix2)
    return np.array([
        [sum(matrix1[i][k] * matrix2[k][j] for k in range(len(matrix2[0])))
         for j in range(len(matrix2[0]))]
        for i in range(len(matrix1))
    ])

def scalar(vector1, vector2):
    assert len(vector1) == len(vector2)
    return sum(x * y for x, y in zip(vector1, vector2))

def norm(vector):
    return np.sqrt(sum(x**2 for x in vector))

def generate_sim_matrix(n=4, a=-15, b=15):
    A = np.zeros(shape=(n, n))
    for i in range(n):
        for j in range(i, n):
            value = random.uniform(a, b)
            A[i][j] = value
            A[j][i] = value
    return A


def gerchgorin_intervals(A):
    intervals = []
    for i in range(len(A)):
        center = A[i][i]
        radius = sum(abs(A[i][j]) for j in range(len(A)) if j != i)
        intervals.append([center - radius, center + radius])

    intervals.sort(key=lambda x:x[0])
    merged = [intervals[0]]
    for i in range(1, len(intervals)):
        current_interval = intervals[i]
        previous_interval = merged[-1]
        if current_interval[0] <= previous_interval[1]:
            merged[-1] = [previous_interval[0], max(previous_interval[1], current_interval[1])]
        else:
            merged.append(current_interval)

    return merged

def binary_search_roots(intervals, f, delta=1e-3):
    lambdas = []
    for interval in intervals:
        left = interval[0]
        right = interval[1]
        f_left = f(left)
        assert f_left * f(right) < 0
        x = (left + right) / 2
        f_x = f(x)
        while abs(f_x) > delta:
            if f_left * f_x < 0:
                right = x
            else:
                left = x
                f_left = f(left)
            x = (left + right) / 2
            if x == left or x == right:
                break
            f_x = f(x)
        lambdas.append(x)
    return lambdas


def binary_search_intervals(intervals, f, delta=0.05):
    search_intervals = []
    for interval in intervals:
        left = interval[0]
        right = interval[1]
        delta_x = left + delta
        while delta_x <= right:
            f_left = f(left)
            f_delta = f(delta_x)
            if f_left * f_delta < 0:
                search_intervals.append([left, delta_x])
                left = delta_x
                delta_x = left + delta
            else:
                delta_x += delta
    return search_intervals

def check_gerchgorin(intervals, eigs):
    print(eigs)
    print(intervals)
    for eig in eigs:
        interval_found = False
        for interval in intervals:
            if interval[0] <= eig <= interval[1]:
                interval_found = True
                break
        if not interval_found:
            print(f"Eig: {eig} not in gerchgoin circles")
            return
    print("All eigs in gerchgoin circles")


def is_sym_matrix(A):
    for i in range(len(A)):
        for j in range(i, len(A)):
            if A[i][j] != A[j][i]:
                return False
    return True

def krylov_eig_vectors(A, Y, coefs, eigs):
    n = len(A)
    Q = np.zeros(shape=(n, n))
    for i in range(n):
        for j in range(n):
           Q[j][i] = 1 if j == 0 else eigs[i] * Q[j - 1][i] + coefs[j - 1]
    vectors = []
    for i in range(n):
        vector = deepcopy(Y[0])
        for j in range(1, n):
            vector += Q[j][i] * Y[j]
        vectors.append(vector / norm(vector))
    return vectors


def krylov(A):
    n = len(A)
    y = np.random.uniform(-10, 10, n)
    Y = [y]
    for i in range(n - 1):
        y = mul_matrix_by_vector(A, y)
        Y.append(y)
    b = mul_matrix_by_vector(A, y)
    Y = np.array(list(reversed(Y)))
    P = Y.T
    coefs = np.linalg.solve(P, b)
    return coefs, Y

def eig(A):
    assert len(A) == len(A[0])
    assert is_sym_matrix(A)
    n = len(A)
    intervals = gerchgorin_intervals(A)
    trace = sum(A[i][i] for i in range(n))
    coefs, Y = krylov(A)
    coefs = list(map(lambda x: x * -1, coefs))
    f = lambda x: np.polyval([1] + coefs, x)
    search_intervals = binary_search_intervals(intervals, f)
    assert len(search_intervals) == n
    eigs = binary_search_roots(search_intervals, f)
    check_gerchgorin(intervals, eigs)
    print(f"Viet theorem:\nSum = {sum(eigs)}\nTrace = {trace}")
    eig_vectors = krylov_eig_vectors(A, Y, coefs, eigs)
    print("Ortnorm eig vectors:")
    print('Norms:')
    for vector in eig_vectors:
        print(norm(vector))
    print("Scalar prods:")
    for i in range(n - 1):
        for j in range(i + 1, n):
            print(scalar(eig_vectors[i], eig_vectors[j]))
    return np.array(eigs), eig_vectors



In [10]:
A = [
    [2.2, 1, 0.5, 2],
    [1, 1.3, 2, 1],
    [0.5, 2, 0.5, 1.6],
    [2, 1, 1.6, 2]
]

print(eig(A))

[-1.4200744628906299, 0.22260742187499594, 1.5453979492187466, 5.652032089233383]
[[-3.5999999999999996, 6.6]]
All eigs in gerchgoin circles
Viet theorem:
Sum = 5.999962997436495
Trace = 6.0
Ortnorm eig vectors:
Norms:
1.0
1.0
0.9999999999999999
1.0
Scalar prods:
8.484202662908946e-05
-3.783197982212827e-05
1.3115570819954847e-05
1.868364355062102e-06
3.4915094466736463e-06
-4.116479529900352e-06
(array([-1.42007446,  0.22260742,  1.54539795,  5.65203209]), [array([-0.22206997,  0.51584535, -0.75728481,  0.33332899]), array([-0.52191192, -0.45488003,  0.15343599,  0.7050883 ]), array([-0.62893738,  0.57256698,  0.48565903, -0.20184186]), array([-0.53173608, -0.44619405, -0.40881551, -0.59248417])])


In [11]:
A = generate_sim_matrix(n=10)

print(eig(A))

[-35.599798022443466, -29.76655797775262, -24.580232160718367, -8.343596043300298, -0.979091477167227, 7.483243592367319, 17.040811162715478, 19.68815017286262, 36.78165671404301, 46.61041670088662]
[[-73.07715609223763, 94.00696504788776]]
All eigs in gerchgoin circles
Viet theorem:
Sum = 28.33500266149308
Trace = 28.335002661494983
Ortnorm eig vectors:
Norms:
1.0
1.0
1.0
1.0000000000000002
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
Scalar prods:
1.719180353632055e-13
4.2511133502287635e-14
5.5441762292218755e-14
2.9217878116938323e-12
3.145816940275381e-13
7.971141108287227e-14
2.707001289792288e-13
1.124100812432971e-15
-1.0824674490095276e-15
1.2112186253965262e-13
-6.589173651150304e-14
-1.83249249108286e-12
-1.6968371152614736e-13
-3.6247047030535384e-14
-1.4171996909340123e-13
2.1510571102112408e-14
-2.2620794126737565e-14
6.542856534341723e-14
1.6186383830496531e-12
3.0415947538386945e-13
1.0153358188935702e-13
2.1536938898947255e-13
4.342012860369948e-15
1.1475195793586579e-15
-7.

In [12]:
A = generate_sim_matrix(n=20)

print(eig(A))

[-68.3170561261625, -54.83115889552465, -48.235218497547635, -41.76442293137558, -35.090794384664804, -24.634917390352882, -23.119631253906775, -17.97120347119906, -9.459747990219581, -6.680448341370634, -2.977379866283875, 8.90300946126268, 16.76624285491735, 20.709966390271205, 28.774505228801786, 35.66204473712466, 40.300522990275624, 45.36853885311591, 51.6772475777166, 71.24913568999739]
[[-173.0853525422325, 171.13737402739827]]
All eigs in gerchgoin circles
Viet theorem:
Sum = -13.670765365124709
Trace = -13.670765114919064
Ortnorm eig vectors:
Norms:
1.0
1.0
0.9999999999999997
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0
0.9999999999999999
1.0
1.0000000000000002
1.0
0.9999999999999998
1.0
0.9999999999999999
1.0
1.0000000000000002
1.0
Scalar prods:
7.66403607244115e-12
3.968113164443121e-11
5.039224454383984e-10
1.445336489380722e-09
8.46135824669747e-08
1.2082540585406365e-07
5.8808752381733775e-09
9.685986443463968e-08
5.432018197054633e-08
4.305597877535261e-08
2